# Data Setup

Downloads and sets up the raw training data for models

### PASCAL data setup

In [1]:
import requests
import tarfile
import os
from pathlib import Path

In [4]:
# directory setup
DATA_PATH = Path("..") / "data"
PASCAL_PARTS_PATH = DATA_PATH / "PASCAL_PARTS"
os.makedirs(PASCAL_PARTS_PATH, exist_ok="true")

In [5]:
# PASCAL_PARTS dataset url
PASCAL_PARTS_URL = "https://roozbehm.info/pascal-parts/trainval.tar.gz"

# download and extract
response = requests.get(PASCAL_PARTS_URL, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path=PASCAL_PARTS_PATH)
print("PASCAL-Parts downloaded")

PASCAL-Parts downloaded


In [10]:
# PASCAL images path
PASCAL_IMAGES_PATH = DATA_PATH / "PASCAL_IMAGES"
os.makedirs(PASCAL_IMAGES_PATH, exist_ok="true")

# PASCAL Images url
PASCAL_IMAGES = "http://host.robots.ox.ac.uk/pascal/VOC/voc2010/VOCtrainval_03-May-2010.tar"

# download images
response = requests.get(PASCAL_IMAGES, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|*")
file.extractall(path=PASCAL_IMAGES_PATH)
print("PASCAL-Images downloaded")

PASCAL-Images downloaded


# Contrived Dataset

## Preprocessing

In [1]:
import DataReader
import os
from pathlib import Path
import json
import pickle

In [2]:
root_dir = Path("..") / "data" / "dataset_3"

for json_ann in os.listdir(root_dir / "annotations"):
    with open(root_dir / "annotations" / json_ann) as input_f:
        ann = DataReader.dictToAnnotation(json.load(input_f))
        
    with open(root_dir / "treeAnnotations" / (json_ann.split(".")[0]+".pkl"), "wb") as out_f:
        pickle.dump(ann, out_f)

In [3]:
# reading annotation
with open(root_dir / "treeAnnotations" / "1.pkl", "rb") as f:
    print(pickle.load(f))

('0', array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), [('0,0', array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), [('0,0,0', array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., Fals